In [1]:
from nltk.corpus import gutenberg
from nltk.text import Text
from nltk import word_tokenize
import pandas as pd
import os

#if os.getcwd()[-1] in '0123456789':
#    path_parent = os.path.dirname(os.getcwd())
#    os.chdir(path_parent)

paper_abstracts = pd.read_csv("paper_abstract_dataset.csv")
paper_abstracts = paper_abstracts[paper_abstracts['Abstract'].notna()].reset_index(drop=True)

In [2]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

from string import punctuation
punctuation = list(punctuation)

from nltk.stem import PorterStemmer
ps = PorterStemmer()

import re

tokens = []

def get_tokens(abstract):
    abstract = abstract.lower()
    tokens_raw = word_tokenize(abstract)
    
    tokens = []
    for token in tokens_raw:
        if bool(re.search(r'\d+', token)):
            continue
        elif token in punctuation or token in stopwords:
            continue
        
        
        
        ps.stem(token)
        tokens.append(token)
    
    return tokens

In [3]:
paper_abstracts.insert(2, "Tokens", "")

## DO NOT RUN

In [4]:
for i,row in enumerate(paper_abstracts['Abstract']):
    if i % 1000 == 0:
        print(i)
    paper_abstracts["Tokens"][i] = get_tokens(row)

0


KeyboardInterrupt: 

In [5]:
paper_abstracts.to_csv("paper_abstract_dataset_with_tokens.csv")

## RUN FROM HERE

In [5]:
paper_abstracts = pd.read_csv("paper_abstract_dataset_with_tokens.csv")

/var/folders/pl/7f1cr2657p3bnpdt3bnm_9w00000gn/T/ipykernel_2116/3044418545.py:1: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  paper_abstracts = pd.read_csv("paper_abstract_dataset_with_tokens.csv")


In [9]:
from nltk import *
bigrams_list = [list(bigrams(row)) for row in paper_abstracts["Tokens"]]

In [31]:
import numpy as np
import scipy
import pickle
p_values = []
for i, bigrams in enumerate(bigrams_list):
    len_bigrams = len(bigrams)
    bigrams1 = np.array([bg[0] for bg in bigrams])
    bigrams2 = np.array([bg[1] for bg in bigrams])
    for bigram in bigrams:
        first_value_true = bigram[0] == bigrams1
        second_value_true = bigram[1] == bigrams2
        contingency_table = np.empty((2,2))
        contingency_table[0,0] = np.sum(np.logical_and(first_value_true, second_value_true))
        contingency_table[1,0] = np.sum(np.logical_and(first_value_true, np.logical_not(second_value_true)))
        contingency_table[0,1] = np.sum(np.logical_and(np.logical_not(first_value_true), second_value_true))
        contingency_table[1,1] = np.sum(np.logical_and(np.logical_not(first_value_true), np.logical_not(second_value_true)))
        p_value = scipy.stats.chi2_contingency(contingency_table)
        p_values.append([i, bigram, p_value[1], len_bigrams])
    if i % 10000 == 0:
        print(i)
        with open('p_values.pickle', 'wb') as handle:
            pickle.dump(p_value, handle, protocol=pickle.HIGHEST_PROTOCOL)
        



0
